In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cell 1 ▶ Install required packages
!pip install timm torchvision tifffile imagecodecs --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 MB 49.9 MB/s eta 0:00:00


In [ ]:
# Cell 2 ▶ Consolidated imports
import os
import math
import time

import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

import timm
from torchvision import transforms, models

from tifffile import imread

In [ ]:
# Cell 1 ▶ Dataset (224×224 targets, no down-scaling)
class LSTDataset(Dataset):
    def __init__(self, df, patches_dir, weather_cols):
        self.df           = df.reset_index(drop=True)
        self.patches_dir  = patches_dir
        self.weather_cols = weather_cols
        self.transform    = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),          # resize image
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std =[0.229, 0.224, 0.225]),
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row  = self.df.loc[idx]
        arr  = imread(os.path.join(self.patches_dir, row["filename"])
                     ).astype(np.float32)            # (4,H,W)

        # --- inputs -------------------------------------------------
        img_np = arr[[1,2,3]].transpose(1,2,0).astype(np.uint8)
        img    = self.transform(img_np)              # [3,224,224]

        # --- target (LST) at *full* 224×224 --------------------------
        lst    = arr[0]                              # (H,W)
        lst    = torch.tensor(lst, dtype=torch.float32).unsqueeze(0)
        lst    = F.interpolate(lst.unsqueeze(0), size=(224,224),
                               mode='bilinear', align_corners=False
                              ).squeeze(0)           # [1,224,224]

        # --- meteorology vector -------------------------------------
        weather = torch.tensor(
            row[self.weather_cols].values.astype(np.float32)
        )
        return img, weather, lst


In [ ]:
# define which meteorological columns to pull
weather_cols = [
    "air_temp_C",
    "dew_point_C",
    "relative_humidity_percent",
    "wind_speed_m_s",
    "precipitation_in",
]

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/PatchedOutput/tiff_with_meteo.csv")
for col in weather_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")
df = df.dropna(subset=weather_cols + ["filename"]).reset_index(drop=True)

patches_dir = "/content/drive/MyDrive/PatchedOutput_Cleaned"
dataset     = LSTDataset(df, patches_dir, weather_cols)
train_sz    = int(0.8 * len(dataset))
val_sz      = len(dataset) - train_sz
train_ds, val_ds = random_split(dataset, [train_sz, val_sz])

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True,  num_workers=0, pin_memory=False)
val_loader   = DataLoader(val_ds,   batch_size=4, shuffle=False, num_workers=0, pin_memory=False)

In [ ]:
import re

# 1) Read in the master CSV
df = pd.read_csv("/content/drive/MyDrive/PatchedOutput/tiff_with_meteo.csv")

# 2) Convert weather columns to numeric & drop NaNs
for col in weather_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")
df = df.dropna(subset=weather_cols + ["filename"]).reset_index(drop=True)

# 3) Extract patch_id (“r1152_c672”) and date (“2022-07-12”) from filename
def parse_patch_id(fn):
    m = re.search(r'(r\d+_c\d+)', fn)
    return m.group(1) if m else None

def parse_date(fn):
    m = re.search(r'_(\d{4}-\d{2}-\d{2})_', fn)
    return pd.to_datetime(m.group(1)) if m else pd.NaT

df['patch_id'] = df['filename'].map(parse_patch_id)
df['date']     = df['filename'].map(parse_date)

# 4) Drop any rows where parsing failed
df = df.dropna(subset=["patch_id", "date"]).reset_index(drop=True)

print(f"Loaded {len(df)} rows; found {df['patch_id'].nunique()} unique patches")

Loaded 14290 rows; found 174 unique patches


In [ ]:
class LSTTimeSeriesDataset(torch.utils.data.Dataset):
    def __init__(self, df, patches_dir, weather_cols, Δt_days=16):
        self.df = df.copy()
        self.patches_dir = patches_dir
        self.weather_cols = weather_cols
        self.Δt = np.timedelta64(int(Δt_days), 'D')

        # Group by patch_id, sorted in time
        self.groups = {
            pid: g.sort_values('date').reset_index(drop=True)
            for pid, g in self.df.groupby('patch_id')
        }

        # Build index of valid (pid, i) pairs where date[i+1] - date[i] == Δt
        self.index = []
        for pid, g in self.groups.items():
            for i in range(len(g)-1):
                if (g.loc[i+1,'date'] - g.loc[i,'date']) == self.Δt:
                    self.index.append((pid, i))

        # Reuse your standard image transform
        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485,0.456,0.406],
                                 std=[0.229,0.224,0.225])
        ])

    def __len__(self):
        return len(self.index)

    def __getitem__(self, idx):
        pid, i = self.index[idx]
        g       = self.groups[pid]

        # --- t0: image + weather + lst
        row0 = g.loc[i]
        arr = imread(os.path.join(self.patches_dir, row0.filename))
        img_np = arr[[1,2,3]].transpose(1,2,0).astype(np.uint8)
        img    = self.transform(img_np)              # [3,224,224]

        w0 = torch.tensor(
            row0[self.weather_cols].values.astype(np.float32),
            dtype=torch.float32
        )                                                     # [len(weather_cols)]

        lst0    = arr[0]                              # (H,W)
        lst0    = torch.tensor(lst0, dtype=torch.float32).unsqueeze(0)
        lst0    = F.interpolate(lst0.unsqueeze(0), size=(224,224),
                               mode='bilinear', align_corners=False
                              ).squeeze(0)           # [1,224,224]                                          # [1,224,224]

        # --- t1: only LST target
        row1 = g.loc[i+1]
        arr1 = imread(os.path.join(
            self.patches_dir,
            row1.filename
        ))
        lst1_np = arr1[0]                                    # (H,W)
        lst1 = torch.tensor(lst1_np, dtype=torch.float32).unsqueeze(0)
        lst1 = F.interpolate(
            lst1.unsqueeze(0),
            size=(224,224),
            mode='bilinear', align_corners=False
        ).squeeze(0)                                          # [1,224,224]

        return img, w0, lst0, lst1


In [ ]:
ts_dataset = LSTTimeSeriesDataset(df, patches_dir, weather_cols)
train_sz = int(0.8 * len(ts_dataset))
train_ds, val_ds = random_split(ts_dataset, [train_sz, len(ts_dataset)-train_sz])

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=4, shuffle=False)

In [ ]:
# Cell 4 ▶ ViT + weather → 224×224 decoder (with α)
import math, torch.nn as nn, timm

class PretrainedViTLSTModel(nn.Module):
    def __init__(self,
                 weather_dim=5,
                 hidden_dim=768,
                 vit_name="vit_base_patch16_224",
                 num_layers=2,
                 num_heads=8):
        super().__init__()
        # --- frozen ViT backbone ---
        self.vit = timm.create_model(vit_name, pretrained=True, num_classes=0)
        for p in self.vit.parameters():
            p.requires_grad = False

        # --- project weather → token ---
        self.weather_proj = nn.Linear(weather_dim, hidden_dim)

        # --- tiny transformer for fusion ---
        enc = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim * 4,
            dropout=0.1
        )
        self.transformer = nn.TransformerEncoder(enc, num_layers)

        # --- decoder: 14→224 via conv-transpose ---
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(hidden_dim, hidden_dim//2, 2, 2),
            nn.BatchNorm2d(hidden_dim//2), nn.ReLU(inplace=True),

            nn.ConvTranspose2d(hidden_dim//2, hidden_dim//4, 2, 2),
            nn.BatchNorm2d(hidden_dim//4), nn.ReLU(inplace=True),

            nn.ConvTranspose2d(hidden_dim//4, hidden_dim//8, 2, 2),
            nn.BatchNorm2d(hidden_dim//8), nn.ReLU(inplace=True),

            nn.ConvTranspose2d(hidden_dim//8, 1, 2, 2)
        )

        # --- learnable Newton-cooling coefficient α ---
        self.alpha = nn.Parameter(torch.tensor(0.01, dtype=torch.float32))

    def forward(self, images, weather):
        feats   = self.vit.forward_features(images)  # [B,197,768]
        cls_tok = feats[:, :1]                       # [B,1,768]
        patch_t = feats[:, 1:]                       # [B,196,768]

        w_tok   = self.weather_proj(weather).unsqueeze(1)  # [B,1,768]
        tokens  = torch.cat([patch_t, w_tok, cls_tok], 1)  # [B,198,768]

        t = self.transformer(tokens.permute(1,0,2)).permute(1,0,2)
        patch_out = t[:, :-2, :]                       # drop weather+CLS

        B, N, D = patch_out.shape  # N=196
        G = int(math.sqrt(N))      # =14
        x = patch_out.transpose(1,2).view(B, D, G, G)   # [B,768,14,14]
        return self.deconv(x)                          # [B,1,224,224]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = PretrainedViTLSTModel(
    weather_dim=len(weather_cols),
    hidden_dim=768,
    vit_name="vit_base_patch16_224",
    num_layers=2,
    num_heads=8
).to(device)

# unfreeze last ViT blocks:
for name, param in model.vit.named_parameters():
    if any(layer in name for layer in ["blocks.10", "blocks.11", "norm"]):
        param.requires_grad = True

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
opt       = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4, weight_decay=1e-2
)
loss_fn   = nn.SmoothL1Loss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    opt, mode='min', factor=0.5, patience=3, verbose=True
)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
# Cell 0 ▶ install LZW support
!pip install imagecodecs --quiet


In [ ]:
from tifffile import imread


In [ ]:
from tqdm import tqdm
import os
from pathlib import Path
import torch

save_dir = Path("/content/drive/MyDrive/Model_vit_mlp_transformer_30m_Checkpoints")
save_dir.mkdir(parents=True, exist_ok=True)

num_epochs = 10
start_ep   = 0

λ_phys   = 1.0     # physics‐loss weight
Δt_days  = 16.0    # 16‐day revisit

for epoch in range(start_ep, num_epochs):
    # — TRAIN —
    model.train()
    total_data_loss = 0.0
    total_phys_loss = 0.0

    for imgs, weather, lst0, lst1 in tqdm(train_loader, desc=f"Epoch {epoch+1:02d} Train"):
        imgs, weather, lst0, lst1 = (
            imgs.to(device), weather.to(device),
            lst0.to(device), lst1.to(device)
        )

        opt.zero_grad()
        pred1 = model(imgs, weather)  # [B,1,224,224]

        # 1) Data‐fit loss (per‐pixel)
        data_loss = loss_fn(pred1, lst1)

        # 2) Physics loss on spatial means
        lst0_mean  = lst0.mean(dim=[2,3])        # [B,1]
        pred1_mean = pred1.mean(dim=[2,3])      # [B,1]
        T_air0     = weather[:, 0].view(-1,1)   # [B,1]  (air_temp_C)

        phys_residual = (pred1_mean - lst0_mean) / Δt_days \
                        - model.alpha * (T_air0 - lst0_mean)
        phys_loss = phys_residual.pow(2).mean()

        # 3) Combined loss
        loss = data_loss + λ_phys * phys_loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step()

        total_data_loss += data_loss.item() * imgs.size(0)
        total_phys_loss += phys_loss.item() * imgs.size(0)

    # epoch‐level metrics
    avg_data_loss = total_data_loss / len(train_loader.dataset)
    avg_phys_loss = total_phys_loss / len(train_loader.dataset)
    train_rmse    = (avg_data_loss + λ_phys * avg_phys_loss) ** 0.5

    # — VALIDATION —
    model.eval()
    val_data_loss = 0.0
    with torch.no_grad():
        for imgs, weather, lst0, lst1 in tqdm(val_loader, desc=f"Epoch {epoch+1:02d} Val"):
            imgs, weather, lst0, lst1 = (
                imgs.to(device), weather.to(device),
                lst0.to(device), lst1.to(device)
            )
            pred1 = model(imgs, weather)
            batch_loss = loss_fn(pred1, lst1).item()
            val_data_loss += batch_loss * imgs.size(0)

    val_rmse = (val_data_loss / len(val_loader.dataset)) ** 0.5

    # — LR scheduler & checkpoint —
    scheduler.step(val_data_loss)
    print(f"Epoch {epoch+1:02d} ▶ Train RMSE: {train_rmse:.3f} | Val RMSE: {val_rmse:.3f}")

    ckpt = save_dir / f"pinn_epoch{epoch+1:02d}.pt"
    torch.save({
        'epoch': epoch+1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': opt.state_dict(),
        'train_rmse': train_rmse,
        'val_rmse': val_rmse,
        'alpha': model.alpha.detach().cpu().item()
    }, ckpt)
    print(f"✅ Saved checkpoint: {ckpt}")

print("✅ Training finished")


Epoch 01 Train:  47%|████▋     | 965/2058 [1:39:54<1:33:30,  5.13s/it]